## Preparation

In [1]:
import geopandas as gpd
import pandas as pd
import os
import glob
import datetime
from funcs.geo import GeoFunctions as gis

In [2]:
# read inputs from spreadsheet
working_dir = os.getcwd()
inputs_dir = os.path.join(working_dir, 'inputs.csv')
inputs_df = pd.read_csv(inputs_dir)

In [3]:
# choose a season
# by index
index = 0
row = inputs_df.iloc[index]
# # by name
# name = '2020_2021_wheat'
# row = inputs_df.loc[inputs_df['season'] == name].iloc[0]

season = row['season']
project_area_dir = row['project_area']
field_area_dir = row['field_area']
start_date = row['start_date']
end_date = row['end_date']

In [4]:
shp_name = os.path.basename(project_area_dir).split('.')[0]
results_dir = os.path.join(working_dir, 'data', season)
products_output_dir = os.path.join(working_dir, "local_data", 'products_outputs', season)

## Products to results data

In [5]:
from funcs.export import product_to_results

In [6]:
gis_in = gis()
eta_tifs = sorted(glob.glob(os.path.join(products_output_dir, "sebal_s2", "eta", "*.tif")))
template = gis_in.get_tif_props(eta_tifs[0])

In [7]:
# sebal_s2 product
time_period = "5days"	# or dekad (10 days) or month
product_to_results(eta_tifs, results_dir, 'eta', time_period, template)

In [ ]:
# WaPOR ETp product
etp_tifs = sorted(glob.glob(os.path.join(products_output_dir, "WaPOR", "WaPOR_data", "L1-RET-E", "*.tif")))
if "L1-RET-E" in os.path.basename(etp_tifs[0]):
	for etp_tif in etp_tifs:
		date = etp_tif.split("_")[-1].split(".")[0]
		new_name = f"etp_{date}.tif"
		os.rename(etp_tif, os.path.join(os.path.dirname(etp_tif), new_name))
else:
	print("already renamed")
etp_tifs = sorted(glob.glob(os.path.join(products_output_dir, "WaPOR", "WaPOR_data", "L1-RET-E", "*.tif")))
time_period = "5days"	# or dekad (10 days) or month
product_to_results(etp_tifs, results_dir, 'etp', time_period, template)

## Kc (ETa/ETp) tifs

In [10]:
from funcs.results_products import kc_tifs
time_period = "5days"	# or dekad (10 days) or month
eta_tifs = sorted(glob.glob(os.path.join(results_dir, time_period, "*", "eta", "*.tif")))
eta_dates = [os.path.basename(i).split(".")[0].split("_")[-1] for i in eta_tifs]
etp_tifs = [tif for tif in sorted(glob.glob(os.path.join(results_dir, time_period, "*", "etp", "*.tif"))) if any(date in tif for date in eta_dates)]
kc_tifs(eta_tifs, etp_tifs, results_dir, time_period)

## Dataframes

In [5]:
from funcs.write_df import write_df_project, write_df_field, fill_et_df

In [6]:
time_period = "5days"	# or dekad (10 days) or month
field_gdf = gpd.read_file(field_area_dir)
project_gdf = gpd.read_file(project_area_dir)
name_attribute = 'id'
days = pd.date_range(start_date, end_date, freq='D')
et_project = pd.DataFrame({"date": days})
et_field = pd.DataFrame({"date": days})

In [7]:
data_name = 'eta'
tifs_dir = sorted(glob.glob(os.path.join(results_dir, time_period, "*", data_name, "*.tif")))
et_project = write_df_project(project_gdf, et_project, tifs_dir, data_name)
data_name = 'etp'
tifs_dir = sorted(glob.glob(os.path.join(results_dir, time_period, "*", data_name, "*.tif")))
et_project = write_df_project(project_gdf, et_project, tifs_dir, data_name)
data_name = 'kc'
tifs_dir = sorted(glob.glob(os.path.join(results_dir, time_period, "*", data_name, "*.tif")))
et_project = write_df_project(project_gdf, et_project, tifs_dir, data_name)
fill_et_df(et_project)
csv_dst = os.path.join(results_dir, 'sheets', 'et_project.csv')
os.makedirs(os.path.dirname(csv_dst), exist_ok= True)
et_project.to_csv(csv_dst, index=False)

In [ ]:
data_name = 'eta'
tifs_dir = sorted(glob.glob(os.path.join(results_dir, time_period, "*", data_name, "*.tif")))
et_field = write_df_field(field_gdf, name_attribute, et_field, tifs_dir, data_name)
data_name = 'etp'
tifs_dir = sorted(glob.glob(os.path.join(results_dir, time_period, "*", data_name, "*.tif")))
et_field = write_df_field(field_gdf, name_attribute, et_field, tifs_dir, data_name)
data_name = 'kc'
tifs_dir = sorted(glob.glob(os.path.join(results_dir, time_period, "*", data_name, "*.tif")))
et_field = write_df_field(field_gdf, name_attribute, et_field, tifs_dir, data_name)
fill_et_df(et_field)
csv_dst = os.path.join(results_dir, 'sheets', 'et_field.csv')
os.makedirs(os.path.dirname(csv_dst), exist_ok= True)
et_field.to_csv(csv_dst, index=False)

In [6]:
# et_project = pd.read_csv(os.path.join(results_dir, 'sheets', 'et_project.csv'))
# et_project['date'] = pd.to_datetime(et_project['date'])
# et_field = pd.read_csv(os.path.join(results_dir, 'sheets', 'et_field.csv'))
# et_field['date'] = pd.to_datetime(et_field['date'])

## Period images

In [5]:
# open et_project if exists
et_project = pd.read_csv(os.path.join(results_dir, 'sheets', 'et_project.csv'))
et_project['date'] = pd.to_datetime(et_project['date'])
gis_in = gis()
eta_tifs = sorted(glob.glob(os.path.join(products_output_dir, "sebal_s2", "eta", "*.tif")))
template = gis_in.get_tif_props(eta_tifs[0])

In [ ]:
from funcs.results_products import period_tifs
time_period = "5days"	# or dekad (10 days) or month
data_name = "eta"
period_tifs(results_dir, et_project, data_name, template, time_period)